###  셀레니움 크롤러 메뉴얼
____

In [ ]:
# 추가 패키지 설치
# !pip install supabase # 수파베이스 SDK 설치
# !pip install selenium # 헤드리스 브라우저를 위한 테스트 자동화 툴
# !pip install beautifulsoup4 # html 파싱 툴
# !pip install tqdm # 프로그래스 표시 

* 크롬 브라우저 버전 확인
  * 크롬 화면 우상단 ... 메뉴 버튼 클릭 > 설정 클릭 > 우하단 Chrome 정보 클릭
  
<img src="./images/chrome1.png" width="800">

___


<img src="./images/chrome2.png" width="800">

124.0 버전 확인 (여러분은 다른 버전일수 있음)



* 크롬 드라이버 다운로드
  * 아래 코드로 자신의 운영체제 및 아키텍처 확인
  * https://googlechromelabs.github.io/chrome-for-testing/#stable 
  
  <img src="./images/chrome3.png" width="800">  


In [3]:
import platform
import sys, os, requests, zipfile

# 운영체제 및 아키텍처 확인
os_name = platform.system().lower()
architecture = platform.machine()

if os_name == 'darwin':
    if architecture == 'arm64':
        print("운영체제: macOS, 아키텍처: ARM64")
    elif architecture == 'x86_64':
        print("운영체제: macOS, 아키텍처: x64")
elif os_name == 'windows':
    if sys.maxsize > 2**32:
        print("운영체제: Windows, 아키텍처: 64-bit")
    else:
        print("운영체제: Windows, 아키텍처: 32-bit")
else:
    print(f"운영체제: {os_name}, 아키텍처: {architecture}")


운영체제: macOS, 아키텍처: ARM64


In [4]:
chrome_driver_url = 'https://storage.googleapis.com/chrome-for-testing-public/124.0.6367.91/mac-arm64/chromedriver-mac-arm64.zip'

* 크롬 드라이버 설치 및 작동해보기

In [9]:
# 다운로드
os.makedirs('./driver', exist_ok=True)
with requests.get(chrome_driver_url) as response:
    with open('./driver/chromedriver.zip', 'wb') as file:
        file.write(response.content)


# 압축해제
with zipfile.ZipFile('./driver/chromedriver.zip') as zip_ref:
    zip_ref.extractall('./driver')
os.remove('./driver/chromedriver.zip')

In [5]:
from glob import glob

driver_path = None
if os_name == 'darwin': # 맥 사용자
    driver_path = glob('./driver/**/chromedriver', recursive=True)[0]
else: # 윈도우 사용자
    driver_path = glob('./driver/**/chromedriver.exe', recursive=True)[0]

driver_path

'./driver/chromedriver-mac-arm64/chromedriver'

In [16]:
# 드라이버 파일의 권한을 확인하고 수정
if os_name == 'darwin':  # 맥 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가
else:  # 윈도우 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
service = Service(executable_path=driver_path)
chrome_options =  webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')

driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://mba.kookmin.ac.kr/mba/ai-data/ai-data-intro.do') # 국민대학교 경영대학원 AI빅데이터 전공소개 페이지



In [10]:
# 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [27]:
# 사이트 제목
soup.select_one('title').text



'전공소개 | 국민대학교 경영대학원'

* 크롤링해보기
  * 크롬 우상단 ... 버튼 클릭 > 도구 더보기 > 개발자 도구 로 진입
  * 개발자도구 탭메뉴[요소] 를 통해서 수집하고자 하는 태그(엘리먼트) 확인
  
  <img src="./images/chrome4.png" width="800">



id는 #
class는 .
tag text

In [13]:
title_selector = '.con-info h5' # CSS 셀렉터 문법 (더 배우기 : https://developer.mozilla.org/ko/docs/Web/CSS/CSS_selectors)
content_selector = '.con-info p'

In [14]:
soup.select(title_selector)

[<h5>수요자 중심 맞춤형 커리큘럼</h5>,
 <h5>국내 최고의 AI빅데이터 전문 교수진</h5>,
 <h5>최첨단 실습 환경과 최적의 온·오프라인 강의</h5>,
 <h5>학위와 함께 자격증도 취득, 급변하는 기술은 특강으로!</h5>,
 <h5>국내 최초의 자부심을 반영하는 우수한 교육 성과</h5>,
 <h5>현장실무 중심의 산학협력 교육</h5>]

In [15]:
soup.select(content_selector)

[<p>AI빅데이터 기술의 실용적 활용에 필요한 기초 역량부터 프로젝트 수행까지 교육을 제공합니다.<br/>수업 연한(1년/1.5년/2년) 및 수여 학위(경영학/공학)를 선택할 수 있어 수요자 필요에 맞춤화된 교육 과정을 설계할 수 있습니다.</p>,
 <p>AI빅데이터 분야의 국내 최고 산학연 전문가들로 구성된 전임 교수진이 함께 합니다.<br/>국내 AI빅데이터 분야를 선도하는 산학연 겸임교수, 인적 네트워크를 통해 Program Advisory Group을 운영하여 수준 높은 교육을 제공합니다.</p>,
 <p>최신 GPU가 탑재된 딥러닝 교육 서버를 구축하여 최대 120명이 동시에 딥러닝 실습이 가능한 실습 환경을 제공합니다. 수준 높은 비대면 강의 컨텐츠 제작을 위한 K*STUDIO, 모빌리티가 강화된 최첨단 멀티미디어 계단식 강의실로 최적의 온·오프라인 강의 환경을 경험할 수 있습니다.</p>,
 <p>교과과정과 연계하여 국가공인 자격증인 빅데이터분석기사, ADsP, ADP, SQLD의 취득이 가능합니다. 산학연 전문가의 특강을 수시로 개설하여 정규 교과과정에서 다루기 어려운 최신 기술 및 실무 노하우를 습득할 수 있습니다. 또한, 업무협약을 통해 프로그래밍 등 신입생의 기초학업 능력 배양을 위한 무료 예비학교를 운영하고 있습니다.</p>,
 <p>2013년 데이터사이언스 양성을 위한 국내 최초의 대학원 석사과정을 개설한 이후로 AI빅데이터 분야 국내 최다 졸업생을 배출하였고 현재 졸업생들은 금융, 의료, 마케팅 등 다양한 분야에서 데이터 사이언티스트로 활동하고 있습니다. 이러한 교육 노하우를 인정받아 국내 대표 기업들로부터 AI빅데이터 교육을 위탁받아 진행하고 있습니다.</p>,
 <p>AI빅데이터 기술이나 데이터를 확보하고 있는 기업, 공공기관 과의 업무협약을 통해 현장문제 발굴, 실제 데이터 활용, 실무 노하우 전수 등 산학협력 기반의 실무 중심 교육을 실시하고 있습니다.</p>]

In [18]:
import pandas as pd
scrap_df = pd.DataFrame({
    'title': [tag.text for tag in soup.select(title_selector)],
    'content': [tag.text for tag in soup.select(content_selector)]
})
scrap_df

,title,content
0,수요자 중심 맞춤형 커리큘럼,AI빅데이터 기술의 실용적 활용에 필요한 기초 역량부터 프로젝트 수행까지 교육을 제...
1,국내 최고의 AI빅데이터 전문 교수진,AI빅데이터 분야의 국내 최고 산학연 전문가들로 구성된 전임 교수진이 함께 합니다....
2,최첨단 실습 환경과 최적의 온·오프라인 강의,최신 GPU가 탑재된 딥러닝 교육 서버를 구축하여 최대 120명이 동시에 딥러닝 실...
3,"학위와 함께 자격증도 취득, 급변하는 기술은 특강으로!","교과과정과 연계하여 국가공인 자격증인 빅데이터분석기사, ADsP, ADP, SQLD..."
4,국내 최초의 자부심을 반영하는 우수한 교육 성과,2013년 데이터사이언스 양성을 위한 국내 최초의 대학원 석사과정을 개설한 이후로 ...
5,현장실무 중심의 산학협력 교육,"AI빅데이터 기술이나 데이터를 확보하고 있는 기업, 공공기관 과의 업무협약을 통해 ..."


In [19]:
# 크롤링이 끝난후 반드시 브라우저 자원을 반납해야함.
driver.close()
driver.quit()

#### #3.과제 : 국민대학교 교양대학 교양아고라 [인생책 100 리스트 - 교수님이 추천하신 인생책] 의 책정보를 수집해서 supabase에 저장할 것.
* https://culture.kookmin.ac.kr/agora/100list/

박선아님